# Code for evaluation

### Input:

**Name:** groundtruth  
**Type:** Dataframe  
**Column:** 'main_genres'  
**Datatype:**: List

**Name:** predictions  
**Type:** Series  
**Datatype:**: List

In [ ]:
from os import path
import pandas as pd
import numpy as np

In [ ]:
def compareValues(row):
    return set(row['p']) <= set(row['t'])

def compareEqual(row):
    return set(row['p']) == set(row['t'])

def compareGroundTruth(compare):
    compare['c'] = compare.apply(compareValues, axis = 1)
    
    #Test Code
#     compare['e'] = compare.apply(compareEqual, axis = 1)
#     print sum(compare['c'])
#     print sum(compare['e'])
#     print compare.head()
    
    return compare


Total: Correct means at least 1 of the 

In [ ]:
def evaluatePredictions(compare):
    #Calculate totals
    length = compare['c'].shape[0]
    correct = sum(compare['c'])

    totals = pd.DataFrame(columns=['correct', 'total'])
    totals.loc['total'] = (correct, length)
    totals['accuracy'] = np.around((totals['correct'] / totals['total'] *100.00).astype(np.double),2)
    

    
    #Calculate genre results
    results = pd.DataFrame(columns=['correct', 'total', 'missed', 'wrong'])

    for i, row in compare.iterrows():
        for genre in row['t']:
            if genre not in results.index:
                results.loc[genre] = [0, 0, 0, 0]
            

            #Calculate total
            results.loc[genre]['total'] += 1
                
            #Calculate correct
            if row['c'] & (genre in row['p']):
                results.loc[genre]['correct'] += 1
            
            #Calculate missed
            if genre not in row['p']:
                results.loc[genre]['missed'] += 1
                
        for genre in row['p']:
            if genre not in results.index:
                results.loc[genre] = [0, 0, 0, 0]
           
            #Calculate wrong
            if genre not in row['t']:
                results.loc[genre]['wrong'] += 1

    results['accuracy'] = np.around((results['correct'] / results['total'] *100.00).astype(np.double),2)
    results['missedperc'] = np.around((results['missed'] / results['total'] *100.00).astype(np.double),2)

    results = results.sort_values(ascending=False, by= 'total')
    
    display(totals)
    display(results)
    
    return (totals, results)

In [ ]:
%%time

# t can be:
#    groundtruth['genre1']
#    groundtruth['main_genres']
compare = pd.DataFrame({"p":predictions, "t":groundtruth['main_genres']})

compare = compareGroundTruth(compare)
results = evaluatePredictions(compare)
